In [1]:
#!pip install pyspark

In [2]:

from pyspark import SparkConf, SparkContext
from pyspark.sql import *
from functools import reduce

In [3]:
#connector 설정

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('sql') \
    .set("spark.driver.extraClassPath","jdbc/mysql-connector-java-5.1.44.jar")

sc = SparkContext(conf=conf)

spark = SparkSession(sc)

21/09/08 17:13:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
#db접속정보 설정
                        #[ip주소:port]   #[db명]                 #[유저명]          #[비밀번호]
DB_URL = 'jdbc:mysql://                /       ?useSSL=false&user=        &password=           '


1.가구유형데이터

In [5]:
#데이터파일(csv)을 DataFrame 으로 읽어들인다.
# inferSchema : 데이터를 기반으로 타입을 유추(기본값 : False)
# header : 첫번째 행을 컬럼 값으로 사용

df1 = spark.read.csv("data/2017_2020_가구유형.csv",encoding='cp949', inferSchema = True, header = True)
df1.show(5)

+----------+------------------+-------------+---------------+----+--------+------------+------------+------------+--------------+-------+
|      시도|행정구역별(시군구)|가구주의 성별|가구주의 연령별|시점|일반가구|1세대가구-계|2세대가구-계|3세대가구-계|4세대이상 가구|1인가구|
+----------+------------------+-------------+---------------+----+--------+------------+------------+------------+--------------+-------+
|서울특별시|        서울특별시|           계|           합계|2017| 3813260|      614037|     1775431|      173908|          1297|1180540|
|서울특별시|        서울특별시|           계|           합계|2018| 3839766|      626988|     1740777|      164734|          1121|1229421|
|서울특별시|        서울특별시|           계|           합계|2019| 3896389|      645266|     1710502|      155420|          1020|1299787|
|서울특별시|        서울특별시|           계|           합계|2020| 3982290|      665193|     1691450|      142292|           767|1390701|
|서울특별시|            종로구|           계|           합계|2017|   62372|        9986|       24272|        2839|            23|  23638|
+----------+

In [6]:
#컬럼정보 확인
df1.printSchema

<bound method DataFrame.printSchema of DataFrame[시도: string, 행정구역별(시군구): string, 가구주의 성별: string, 가구주의 연령별: string, 시점: int, 일반가구: int, 1세대가구-계: int, 2세대가구-계: int, 3세대가구-계: int, 4세대이상 가구: string, 1인가구: int]>

In [7]:
# DataFrame 타입인 것을 확인
print(type(df1))

<class 'pyspark.sql.dataframe.DataFrame'>


In [8]:
#전체데이터 DB에 저장

df1 \
    .write \
    .format('jdbc') \
    .mode('append') \
    .option("url", DB_URL) \
    .option("dbtable", "family2017_2020") \
    .save()

2.건축연도데이터

In [9]:
#2017_2020_시군구_건축연도 : 17,18(시점)
df2_1 = spark.read.csv("data/2017_2020_시군구_건축연도_1718.csv",encoding='cp949', inferSchema = True, header = True)
df2_1.show(5)

+----------+------------------+----+-------------+--------+------+------+------+------+------+------+------+------+------+------+------+-----------+-----------+-----------+-----------+-----------+
|      시도|행정구역별(시군구)|시점|주택의 종류별|연면적별|2020년|2019년|2018년|2017년|2016년|2015년|2014년|2013년|2012년|2011년|2010년|2005~2009년|2000~2004년|1990~1999년|1980~1989년|1979년 이전|
+----------+------------------+----+-------------+--------+------+------+------+------+------+------+------+------+------+------+------+-----------+-----------+-----------+-----------+-----------+
|서울특별시|        서울특별시|2017|           계|    합계|     0|     0|     0| 52138| 85696| 71833| 81708| 63132| 67466| 68643| 50016|     300432|     596244|     818777|     432556|     178204|
|서울특별시|        서울특별시|2018|           계|    합계|     0|     0| 44662| 61493| 86546| 72230| 81740| 62582| 67502| 68084| 50036|     299832|     594473|     812594|     421439|     170865|
|서울특별시|            종로구|2017|           계|    합계|     0|     0|     0|   484|   30

In [10]:
#컬럼정보 확인
print(df2_1.printSchema,'\n')
# DataFrame 타입인 것을 확인
print(type(df2_1))

<bound method DataFrame.printSchema of DataFrame[시도: string, 행정구역별(시군구): string, 시점: int, 주택의 종류별: string, 연면적별: string, 2020년: int, 2019년: int, 2018년: int, 2017년: int, 2016년: int, 2015년: int, 2014년: int, 2013년: int, 2012년: int, 2011년: int, 2010년: int, 2005~2009년: int, 2000~2004년: int, 1990~1999년: int, 1980~1989년: int, 1979년 이전: int]> 

<class 'pyspark.sql.dataframe.DataFrame'>


In [11]:
#2017_2020_시군구_건축연도 : 시점 19,20
df2_2 = spark.read.csv("data/2017_2020_시군구_건축연도_1920.csv",encoding='cp949', inferSchema = True, header = True)
df2_2.show(5)

+----------+------------------+----+-------------+--------+------+------+------+------+------+------+------+------+------+------+------+-----------+-----------+-----------+-----------+-----------+
|      시도|행정구역별(시군구)|시점|주택의 종류별|연면적별|2020년|2019년|2018년|2017년|2016년|2015년|2014년|2013년|2012년|2011년|2010년|2005~2009년|2000~2004년|1990~1999년|1980~1989년|1979년 이전|
+----------+------------------+----+-------------+--------+------+------+------+------+------+------+------+------+------+------+------+-----------+-----------+-----------+-----------+-----------+
|서울특별시|        서울특별시|2019|           계|    합계|     0| 65363| 73312| 68328| 86910| 71212| 81695| 62927| 67665| 68145| 50248|     299124|     592177|     806133|     400831|     159894|
|서울특별시|            종로구|2019|           계|    합계|     0|   625|   240|  2892|   303|   474|   548|   792|   353|   257|   222|       4503|       9081|      13446|       4515|       8189|
|서울특별시|              중구|2019|           계|    합계|     0|  1261|   719|   818|  

In [12]:
#컬럼정보 확인
print(df2_2.printSchema,'\n')
# DataFrame 타입인 것을 확인
print(type(df2_2))

<bound method DataFrame.printSchema of DataFrame[시도: string, 행정구역별(시군구): string, 시점: int, 주택의 종류별: string, 연면적별: string, 2020년: int, 2019년: int, 2018년: int, 2017년: int, 2016년: int, 2015년: int, 2014년: int, 2013년: int, 2012년: int, 2011년: int, 2010년: int, 2005~2009년: int, 2000~2004년: int, 1990~1999년: int, 1980~1989년: int, 1979년 이전: int]> 

<class 'pyspark.sql.dataframe.DataFrame'>


In [13]:
#시점 17/18 데이터 DB저장
df2_1 \
    .write \
    .format('jdbc') \
    .mode('append') \
    .option("url", DB_URL) \
    .option("dbtable", "building2017_2020_1") \
    .save()

In [14]:
#시점 19/20 데이터 DB저장
df2_2 \
    .write \
    .format('jdbc') \
    .mode('append') \
    .option("url", DB_URL) \
    .option("dbtable", "building2017_2020_2") \
    .save()

In [15]:
#시점 17-20 하나로 합친 데이터 만들기
def unionAll(*dfs):
    return reduce(DataFrame.union, dfs)

df2_3 = unionAll(df2_1, df2_2)  
df2_3.show(3)

+----------+------------------+----+-------------+--------+------+------+------+------+------+------+------+------+------+------+------+-----------+-----------+-----------+-----------+-----------+
|      시도|행정구역별(시군구)|시점|주택의 종류별|연면적별|2020년|2019년|2018년|2017년|2016년|2015년|2014년|2013년|2012년|2011년|2010년|2005~2009년|2000~2004년|1990~1999년|1980~1989년|1979년 이전|
+----------+------------------+----+-------------+--------+------+------+------+------+------+------+------+------+------+------+------+-----------+-----------+-----------+-----------+-----------+
|서울특별시|        서울특별시|2017|           계|    합계|     0|     0|     0| 52138| 85696| 71833| 81708| 63132| 67466| 68643| 50016|     300432|     596244|     818777|     432556|     178204|
|서울특별시|        서울특별시|2018|           계|    합계|     0|     0| 44662| 61493| 86546| 72230| 81740| 62582| 67502| 68084| 50036|     299832|     594473|     812594|     421439|     170865|
|서울특별시|            종로구|2017|           계|    합계|     0|     0|     0|   484|   30

In [16]:
#컬럼정보 확인
print(df2_3.printSchema,'\n')
# DataFrame 타입인 것을 확인
print(type(df2_3))

<bound method DataFrame.printSchema of DataFrame[시도: string, 행정구역별(시군구): string, 시점: int, 주택의 종류별: string, 연면적별: string, 2020년: int, 2019년: int, 2018년: int, 2017년: int, 2016년: int, 2015년: int, 2014년: int, 2013년: int, 2012년: int, 2011년: int, 2010년: int, 2005~2009년: int, 2000~2004년: int, 1990~1999년: int, 1980~1989년: int, 1979년 이전: int]> 

<class 'pyspark.sql.dataframe.DataFrame'>


In [17]:
#시점 17-20 합친 데이터 DB저장
df2_3 \
    .write \
    .format('jdbc') \
    .mode('append') \
    .option("url", DB_URL) \
    .option("dbtable", "building2017_2020_3") \
    .save()

3.성/연령데이터

In [18]:
df3 = spark.read.csv("data/2017_2020_연령22.csv",encoding='cp949', inferSchema = True, header = True)
df3.show(3)

21/09/08 17:13:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+------+------+-------+-------+---------+---------+------+---------+---------+-------+---------+---------+-------+---------+---------+---------+---------+-------+---------+---------+---------+---------+---------+---------+-----------+---------+---------+---------+---------+----------+-----------+--------+----+----+----+----+----+----+----+----+----+----+
|연도|  시도|시군구|0 - 4세|5 - 9세|10 - 14세|15 - 19세|0-19세|20 - 24세|25 - 29세|20-29세|30 - 34세|35 - 39세|30-39세|40 - 44세|45 - 49세|50 - 54세|55 - 59세|40-59세|60 - 64세|65 - 69세|70 - 74세|75 - 79세|60세-79세|80 - 84세|80세 이상25|85 - 89세|85세 이상|90 - 94세|95 - 99세|100세 이상|80세 이상31|  총합계|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|
+----+------+------+-------+-------+---------+---------+------+---------+---------+-------+---------+---------+-------+---------+---------+---------+---------+-------+---------+---------+---------+---------+---------+---------+-----------+---------+---------+---------+---------+----------+-----------+--------+----+----+---

21/09/08 17:13:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 연도, 시도, 시군구, 0 - 4세, 5 - 9세, 10 - 14세, 15 - 19세, 0-19세, 20 - 24세, 25 - 29세, 20-29세, 30 - 34세, 35 - 39세, 30-39세, 40 - 44세, 45 - 49세, 50 - 54세, 55 - 59세, 40-59세, 60 - 64세, 65 - 69세, 70 - 74세, 75 - 79세, 60세-79세, 80 - 84세, 80세 이상, 85 - 89세, 85세 이상, 90 - 94세, 95 - 99세, 100세 이상, 80세 이상, 총합계, , , , , , , , , , 
 Schema: 연도, 시도, 시군구, 0 - 4세, 5 - 9세, 10 - 14세, 15 - 19세, 0-19세, 20 - 24세, 25 - 29세, 20-29세, 30 - 34세, 35 - 39세, 30-39세, 40 - 44세, 45 - 49세, 50 - 54세, 55 - 59세, 40-59세, 60 - 64세, 65 - 69세, 70 - 74세, 75 - 79세, 60세-79세, 80 - 84세, 80세 이상25, 85 - 89세, 85세 이상, 90 - 94세, 95 - 99세, 100세 이상, 80세 이상31, 총합계, _c33, _c34, _c35, _c36, _c37, _c38, _c39, _c40, _c41, _c42
Expected: 80세 이상25 but found: 80세 이상
CSV file: file:///home/lab04/data/2017_2020_연령22.csv


In [19]:
#컬럼정보 확인
print(df3.printSchema,'\n')
# DataFrame 타입인 것을 확인
print(type(df3))

<bound method DataFrame.printSchema of DataFrame[연도: int, 시도: string, 시군구: string, 0 - 4세: int, 5 - 9세: int, 10 - 14세: int, 15 - 19세: int, 0-19세: int, 20 - 24세: int, 25 - 29세: int, 20-29세: int, 30 - 34세: int, 35 - 39세: int, 30-39세: int, 40 - 44세: int, 45 - 49세: int, 50 - 54세: int, 55 - 59세: int, 40-59세: int, 60 - 64세: int, 65 - 69세: int, 70 - 74세: int, 75 - 79세: int, 60세-79세: int, 80 - 84세: int, 80세 이상25: int, 85 - 89세: int, 85세 이상: int, 90 - 94세: int, 95 - 99세: int, 100세 이상: int, 80세 이상31: int, 총합계: double, _c33: string, _c34: string, _c35: string, _c36: string, _c37: string, _c38: string, _c39: string, _c40: string, _c41: string, _c42: string]> 

<class 'pyspark.sql.dataframe.DataFrame'>


In [20]:
#DB저장
df3 \
    .write \
    .format('jdbc') \
    .mode('append') \
    .option("url", DB_URL) \
    .option("dbtable", "age2017_2020") \
    .save()

21/09/08 17:13:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 연도, 시도, 시군구, 0 - 4세, 5 - 9세, 10 - 14세, 15 - 19세, 0-19세, 20 - 24세, 25 - 29세, 20-29세, 30 - 34세, 35 - 39세, 30-39세, 40 - 44세, 45 - 49세, 50 - 54세, 55 - 59세, 40-59세, 60 - 64세, 65 - 69세, 70 - 74세, 75 - 79세, 60세-79세, 80 - 84세, 80세 이상, 85 - 89세, 85세 이상, 90 - 94세, 95 - 99세, 100세 이상, 80세 이상, 총합계, , , , , , , , , , 
 Schema: 연도, 시도, 시군구, 0 - 4세, 5 - 9세, 10 - 14세, 15 - 19세, 0-19세, 20 - 24세, 25 - 29세, 20-29세, 30 - 34세, 35 - 39세, 30-39세, 40 - 44세, 45 - 49세, 50 - 54세, 55 - 59세, 40-59세, 60 - 64세, 65 - 69세, 70 - 74세, 75 - 79세, 60세-79세, 80 - 84세, 80세 이상25, 85 - 89세, 85세 이상, 90 - 94세, 95 - 99세, 100세 이상, 80세 이상31, 총합계, _c33, _c34, _c35, _c36, _c37, _c38, _c39, _c40, _c41, _c42
Expected: 80세 이상25 but found: 80세 이상
CSV file: file:///home/lab04/data/2017_2020_연령22.csv


4.흡연율데이터

In [21]:
df4 = spark.read.csv("data/2017_2020_흡연율.csv",encoding='cp949', inferSchema = True, header = True)
df4.show(3)

+----+----------+----------+------+
|시점|       _c1|행정구역별|데이터|
+----+----------+----------+------+
|2017|서울특별시|서울특별시|  20.0|
|2017|서울특별시|    종로구|  16.5|
|2017|서울특별시|      중구|  20.8|
+----+----------+----------+------+
only showing top 3 rows



21/09/08 17:13:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 시점, , 행정구역별, 데이터
 Schema: 시점, _c1, 행정구역별, 데이터
Expected: _c1 but found: 
CSV file: file:///home/lab04/data/2017_2020_흡연율.csv


In [22]:
#컬럼정보 확인
print(df4.printSchema,'\n')
# DataFrame 타입인 것을 확인
print(type(df4))

<bound method DataFrame.printSchema of DataFrame[시점: int, _c1: string, 행정구역별: string, 데이터: double]> 

<class 'pyspark.sql.dataframe.DataFrame'>


In [23]:
#DB저장
df4 \
    .write \
    .format('jdbc') \
    .mode('append') \
    .option("url", DB_URL) \
    .option("dbtable", "smoking2017_2020") \
    .save()

21/09/08 17:13:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 시점, , 행정구역별, 데이터
 Schema: 시점, _c1, 행정구역별, 데이터
Expected: _c1 but found: 
CSV file: file:///home/lab04/data/2017_2020_흡연율.csv
